In [1]:
import tensorflow as tf
from collections import Counter
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random

In [5]:
#traing data
real_images_path = 'out_real_ryv'
fake_images_path = 'out_fake_ryv'

images = []
labels = []

# Load real images
for filename in os.listdir(real_images_path)[:2433]:
    if filename.endswith(".png"):  
        img = cv2.imread(os.path.join(real_images_path, filename))
        img = img_to_array(img)
        images.append(img)
        labels.append(1)  

# Load fake images
for filename in os.listdir(fake_images_path)[:2413]:
    if filename.endswith(".png"):  
        img = cv2.imread(os.path.join(fake_images_path, filename))
        img = img_to_array(img)
        images.append(img)
        labels.append(0)  

train_images = np.array(images)
train_labels = np.array(labels)

temp = list(zip(train_images, train_labels))
random.shuffle(temp)
res1, res2 = zip(*temp)
res1, res2 = list(res1), list(res2)

X_train = np.array(res1) / 255.0
y_train= np.array(res2)

# temp=[]
# res1=[]
# res2=[]
# train_images=[]
# train_labels=[]

In [10]:
print(X_train.shape, y_train.shape)

(4846, 128, 32, 3) (4846,)


In [4]:
y_train[:20]

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1])

In [14]:
#testing
real_images_path = 'out_real_ryv'
fake_images_path = 'out_fake_ryv'

images = []
labels = []

# Load real images
for filename in os.listdir(real_images_path)[2433:]:
    if filename.endswith(".png"):  
        img = cv2.imread(os.path.join(real_images_path, filename))
        img = img_to_array(img)
        images.append(img)
        labels.append(1)  

# Load fake images
for filename in os.listdir(fake_images_path)[2413:]:
    if filename.endswith(".png"):  
        img = cv2.imread(os.path.join(fake_images_path, filename))
        img = img_to_array(img)
        images.append(img)
        labels.append(0)  

test_images = np.array(images)
test_labels = np.array(labels)

X_test= np.array(test_images)/255.0
y_test= test_labels

# test_images=[]
# test_labels=[]

In [15]:
print(X_test.shape, y_test.shape)

(1981, 128, 32, 3) (1981,)


In [29]:
model=[]

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(128, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.005)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.005)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_regularizer=l2(0.005)))
model.add(Dropout(0.2))
model.add(Dense(55, activation='relu', kernel_regularizer=l2(0.005)))
model.add(Dropout(0.3))
model.add(Dense(60, activation='relu', kernel_regularizer=l2(0.005)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [31]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 30, 16)       448       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 63, 15, 16)       0         
 2D)                                                             
                                                                 
 batch_normalization_9 (Batc  (None, 63, 15, 16)       64        
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 13, 32)        4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 30, 6, 32)        0         
 g2D)                                                            
                                                      

In [32]:
from tensorflow.keras.optimizers import Adam
custom_optimizer = Adam(learning_rate=0.001)

In [33]:
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy',metrics=['accuracy'])

In [34]:
class EarlyStartTrainingCallback(tf.keras.callbacks.Callback):
    def __init__(self, loss_threshold=0.3, accuracy_threshold=0.95):
        super(EarlyStartTrainingCallback, self).__init__()
        self.loss_threshold = loss_threshold
        self.accuracy_threshold = accuracy_threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        
        val_loss = logs.get('val_loss')
        val_acc = logs.get('val_accuracy')
        if val_loss is not None and val_acc is not None and val_loss < self.loss_threshold and val_acc > self.accuracy_threshold:
            self.model.stop_training = True

# Create the callback
early_start_callback = EarlyStartTrainingCallback(loss_threshold=0.25, accuracy_threshold=0.95)


In [35]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),callbacks=[early_start_callback], epochs=80, batch_size=32)

Epoch 1/80
152/152 [==============================] - 12s 71ms/step - loss: 1.7368 - accuracy: 0.5809 - val_loss: 1.5235 - val_accuracy: 0.5411
Epoch 2/80
152/152 [==============================] - 10s 64ms/step - loss: 1.2943 - accuracy: 0.6830 - val_loss: 1.4785 - val_accuracy: 0.4972
Epoch 3/80
152/152 [==============================] - 10s 64ms/step - loss: 0.9538 - accuracy: 0.7639 - val_loss: 1.2126 - val_accuracy: 0.6371
Epoch 4/80
152/152 [==============================] - 10s 65ms/step - loss: 0.7070 - accuracy: 0.8395 - val_loss: 0.7698 - val_accuracy: 0.7713
Epoch 5/80
152/152 [==============================] - 10s 67ms/step - loss: 0.5303 - accuracy: 0.8898 - val_loss: 0.5275 - val_accuracy: 0.8804
Epoch 6/80
152/152 [==============================] - 11s 71ms/step - loss: 0.4490 - accuracy: 0.9067 - val_loss: 0.4351 - val_accuracy: 0.9011
Epoch 7/80
152/152 [==============================] - 11s 73ms/step - loss: 0.3706 - accuracy: 0.9272 - val_loss: 0.4619 - val_accuracy:

In [83]:
# from tensorflow.keras.models import load_model
# model.save('model_cnn_ppgryv1_train_test.h5')

In [14]:
# import tensorflow as tf
# from tensorflow.keras import layers, models

# # Define the CNN model
# model_dscnn = Sequential()

# # Depth-wise separable convolution layer
# model_dscnn.add(layers.DepthwiseConv2D(3, activation='relu', input_shape=(128,32, 1)))
# model_dscnn.add(layers.BatchNormalization())
# model_dscnn.add(layers.SeparableConv2D(32, (3, 3), activation='relu'))
# model_dscnn.add(layers.BatchNormalization())
# model_dscnn.add(layers.MaxPooling2D((2, 2)))

# # Another depth-wise separable convolution layer
# model_dscnn.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
# model_dscnn.add(layers.BatchNormalization())
# model_dscnn.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
# model_dscnn.add(layers.BatchNormalization())
# model_dscnn.add(layers.MaxPooling2D((2, 2)))
# model_dscnn.add(Flatten())
# model_dscnn.add(layers.Dense(128, activation='relu'))
# model_dscnn.add(layers.Dropout(0.5))
# model_dscnn.add(layers.Dense(1, activation='sigmoid'))  # 2 classes for binary classification

# # Compile the model
# model_dscnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Print the model summary
# model_dscnn.summary()


In [15]:
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=10)

In [36]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

62/62 [==============================] - 1s 20ms/step - loss: 0.2444 - accuracy: 0.9551
Test loss: 0.24435880780220032
Test accuracy: 0.9550731778144836


In [43]:
import cv2
import numpy as np
t1=cv2.imread('out_real_ryv/702_1.png')
t1=t1/255.0
batch_of_images = np.array([t1]*32)
predictions=model.predict(batch_of_images)
predictions[0]

array([0.999983], dtype=float32)

In [44]:
predicted_labels=[]
original_labels=[]
for i in range(300):
    original_labels.append(0) #fake
for i in range(300):
    original_labels.append(1) #real

In [45]:
d_fake={}
testing_videos_fake=os.listdir('out_fake_ryv')[2416:]
for i in testing_videos_fake:
    d_fake[i[:-6]]=[]
for i in testing_videos_fake:
    img_test=cv2.imread(f'E:/FF++/out_fake_ryv/{i}')
    d_fake[i[:-6]].append(img_test)

In [46]:
d_real={}
testing_videos_fake=os.listdir('out_real_ryv')[2436:]
for i in testing_videos_fake:
    d_real[i[:-6]]=[]
for i in testing_videos_fake:
    img_test=cv2.imread(f'E:/FF++/out_real_ryv/{i}')
    d_real[i[:-6]].append(img_test)

In [47]:
len(d_real), len(d_fake)

(300, 300)

In [48]:
predicted_labels=[]

In [49]:
for i, j in d_fake.items():
    segment_predictions=[]
    for k in j:
        k=k/255.0
        batch_of_images = np.array([k] * 32)
        predictions=model.predict([batch_of_images])
        if(predictions[0][0]>0.5):
            pc=1
        else:
            pc=0
        segment_predictions.append(pc)
    print(segment_predictions)
    if(Counter(segment_predictions)[0]>=Counter(segment_predictions)[1]):
        vid_prediction=0
    else:
        vid_prediction=1
    predicted_labels.append(vid_prediction)

[0, 0]
[0, 1, 1, 0, 0]
[0, 0]
[0, 0, 0]
[1, 0, 0]
[0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0]
[0, 0, 0, 0]
[0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[1, 0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0, 0, 0]
[0, 0]
[0, 0]
[0, 0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0]
[0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0]
[0, 0]
[0, 0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0]
[0, 0]
[0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0]
[1, 1, 0]
[0, 0, 0, 0, 0, 0]
[0, 1, 0]
[0, 0]
[0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0]
[0, 0, 0, 0]
[0, 0]
[0, 0, 0, 0]
[1, 1]
[0, 1]
[0, 0]
[0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0]
[0, 0, 0, 0]
[0, 1, 1]
[0, 1, 0, 0]
[0, 0, 0,

In [50]:
for i, j in d_real.items():
    segment_predictions=[]
    for k in j:
        k=k/255.0
        batch_of_images = np.array([k] * 32)
        predictions=model.predict([batch_of_images])
        if(predictions[0][0]>0.5):
            pc=1
        else:
            pc=0
        segment_predictions.append(pc)
    print(segment_predictions)
    if(Counter(segment_predictions)[0]>=Counter(segment_predictions)[1]):
        vid_prediction=0
    else:
        vid_prediction=1
    predicted_labels.append(vid_prediction)

[1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1]
[1, 1, 1, 1]
[1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1]
[0, 1]
[1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1]
[1, 1, 1]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1, 1, 1]
[1, 1]
[1, 1]
[1, 1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1]
[1, 1]
[1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1]
[1, 1]
[1, 1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1]
[1, 1]
[1, 1, 1]
[1, 1, 1, 1, 1, 1]
[1, 1]
[1, 1, 1]
[1, 1, 1, 1, 1, 1]
[1, 1, 1]
[1, 1]
[1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1]
[1, 1, 1, 1]
[1, 1]
[1, 0, 1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1]
[1, 1, 1, 1]
[0, 1, 1]
[0, 1, 1]
[1, 1,

In [51]:
print(len(predicted_labels), len(original_labels))

600 600


In [52]:
for i, j in zip(predicted_labels,original_labels):
    print(i,j)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [53]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
confusion_matrix(original_labels,predicted_labels)

array([[293,   7],
       [ 15, 285]], dtype=int64)

In [54]:

accuracy = accuracy_score(original_labels, predicted_labels)
print("Accuracy   :", accuracy)
precision = precision_score(original_labels, predicted_labels)
print("Precision :", precision)
recall = recall_score(original_labels, predicted_labels)
print("Recall    :", recall)
F1_score = f1_score(original_labels, predicted_labels)
print("F1-score  :", F1_score)

Accuracy   : 0.9633333333333334
Precision : 0.976027397260274
Recall    : 0.95
F1-score  : 0.9628378378378378


In [55]:
import cv2
import dlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from scipy import signal
from sklearn import preprocessing
import os

In [56]:
def get_ppgs(vid_path):
    ppg_maps=[]
    # Define the input video file path
    video_path = vid_path

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the frames per second (fps) and total frame count
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Set the desired number of frames per segment
    frames_per_segment = 128  # Set to 128 frames per segment

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks (1).dat")  # You need to download this file
    # detector.set_min_detection_confidence(0.5)
    # Initialize variables to keep track of segment number and frames
    segment_number = 1
    frame_count = 0
    ppgmap=np.empty([128,32,3])
    ind=0

    while True:
        #print("hi")
        ret, frame = cap.read()
        if not ret:
            break
            
        frame=cv2.resize(frame,(854,480))
        
        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if(len(detector(gray))==0):
            continue

        face = detector(gray)[0]
        landmarks = predictor(gray, face)

        # Extract the coordinates of the nose (e.g., landmark point 30)
        nose_x = landmarks.part(30).x
        nose_y = landmarks.part(30).y

        # Define the ROI around the nose
        roi_size_width = 64
        roi_size_height = 32
        roi_x = nose_x - roi_size_width//2
        roi_y = nose_y - roi_size_height//2

        # Ensure ROI coordinates are within bounds
        roi_x = max(0, roi_x)
        roi_y = max(0, roi_y)
        roi_x_end = min(frame.shape[1], roi_x + roi_size_width)
        roi_y_end = min(frame.shape[0], roi_y + roi_size_height)

        # Extract the ROI
        roi = frame[roi_y:roi_y_end, roi_x:roi_x_end]
        
        # Calculate subregion size
        subregion_width = roi_size_width // 8  # 8 subregions horizontally
        subregion_height = roi_size_height // 4  # 4 subregions vertically

        subregions_r=np.zeros(32)
        subregions_y=np.zeros(32)
        subregions_v=np.zeros(32)
        k=0
        for i in range(4):
            for j in range(8):
                left = j * subregion_width
                upper = i * subregion_height
                right = (j + 1) * subregion_width
                lower = (i + 1) * subregion_height
                subregion=roi[upper:lower, left:right]

                roi_ycbcr = cv2.cvtColor(subregion, cv2.COLOR_BGR2YCrCb)
                roi_hsv = cv2.cvtColor(subregion, cv2.COLOR_BGR2HSV)
                
                y_comp = np.mean(roi_ycbcr[:, :, 0])  
                v_comp = np.mean(roi_hsv[:, :, 2])  
                rv=np.mean(subregion[:,:,0])
                
                subregions_r[k]=rv
                subregions_y[k]=y_comp
                subregions_v[k]=v_comp
                
                k+=1
                
        ppgmap[ind,:,0]=subregions_r
        ppgmap[ind,:,1]=subregions_y
        ppgmap[ind,:,2]=subregions_v
        #print(ppgmap[ind])
        ind+=1
        frame_count += 1
        #curr_seg.append(frame)
        #print(ppgmap[ind])
        if frame_count == frames_per_segment:
            min_values = np.min(ppgmap, axis=(0, 1))
            max_values = np.max(ppgmap, axis=(0, 1))
            scaled_data = ((ppgmap - min_values) / (max_values - min_values) * 255.0).astype(np.uint8)
            ppg_maps.append(scaled_data)
            segment_number += 1
            ppgmap=np.empty([128,32,3])
            ind=0
            frame_count = 0
    cap.release()
    return ppg_maps

In [57]:
ppgs_for_video=get_ppgs(r"C:\Users\hites\Downloads\deepfake-detection-challenge\train_sample_videos\errocgcham.mp4")
segment_predictions=[]
for k in ppgs_for_video:
    k=k/255.0
    batch_of_images = np.array([k] * 32)
    predictions=model.predict([batch_of_images])
    print(predictions[0][0],end=" ")
    if(predictions[0][0]>0.5):
        pc=1
    else:
        pc=0
    segment_predictions.append(pc)
print(segment_predictions)
if(Counter(segment_predictions)[0]>=Counter(segment_predictions)[1]):
    vid_prediction=0
else:
    vid_prediction=1

print("Prediction from our model: ",vid_prediction)

0.98685056 0.32960775 [1, 0]
Prediction from our model:  0
